In [1]:
#import packages
import pandas as pd
import requests
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import urllib.request, json
import requests
import urllib.parse
import datetime
import nltk
nltk.download([
        "names",
        "stopwords",
        "averaged_perceptron_tagger",
        "vader_lexicon",
        "punkt",])

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package names to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### NYC Open Data:
    
SODA API Urls:

* 311 Call Inquiries: `https://data.cityofnewyork.us/resource/wewp-mm3p.json`

 **Important Column Names:**

   * uniqueid = 'UNIQUE_ID'
   * cat_clnm = 'AGENCY_NAME'
   * subcat_clnm = 'INQUIRY_NAME'
   * subsubcat_clnm = 'BRIEF_DESCRIPTION'
   * datetime_clnm = 'DATE_TIME'


* 311 Web Inquiries: `https://data.cityofnewyork.us/resource/vwpc-kje2.json`

 **Important Column Names:**

   * uniqueid = 'simple_id'
   * cat_clnm = 'service_name'
   * subcat_clnm = 'brief_description'
   * subsubcat_clnm = 'detailed_description'
   * datetime_clnm = 'updated_date'

In [2]:
calls =  'https://data.cityofnewyork.us/resource/wewp-mm3p.json'
web = 'https://data.cityofnewyork.us/resource/vwpc-kje2.json'

uniqueid = 'UNIQUE_ID'
cat_clnm = 'AGENCY_NAME'
subcat_clnm = 'INQUIRY_NAME'
subsubcat_clnm = 'BRIEF_DESCRIPTION'
datetime_clnm = 'DATE_TIME'
startdatetime = '2019-01-01T00:00:00.000'
enddatetime =  '2020-01-01T00:00:00.000'

query = (calls +'?'
    "$select=count(*) as count,%s as cat, %s as subcat, %s as subsubcat"
    "&$where=%s between '"+startdatetime+"' and '"+enddatetime+"'"
    "&$group=cat,subcat,subsubcat"
    "&$order=count DESC"
    "&$limit=500000"
    )%(cat_clnm,subcat_clnm,subsubcat_clnm,datetime_clnm)
query = query.replace(" ", "%20")
print(query)
response = urllib.request.urlopen(query)
data = json.loads(response.read())

https://data.cityofnewyork.us/resource/wewp-mm3p.json?$select=count(*)%20as%20count,AGENCY_NAME%20as%20cat,%20INQUIRY_NAME%20as%20subcat,%20BRIEF_DESCRIPTION%20as%20subsubcat&$where=DATE_TIME%20between%20'2019-01-01T00:00:00.000'%20and%20'2020-01-01T00:00:00.000'&$group=cat,subcat,subsubcat&$order=count%20DESC&$limit=500000


In [6]:
call_comps = pd.DataFrame(data, columns = data[0].keys())
call_comps.head(10)

,count,cat,subcat,subsubcat
0,84545,Department of Finance,Parking Ticket Lookup - Ticket or Plate Number...,Status of a parking ticket. Must know ticket o...
1,76516,3-1-1 Call Center,Service Request Status - General,Status of most Service Requests filed with 311.
2,60523,Department of Finance,Property Tax Account Assistance and Bill Infor...,Help with questions about the property tax bil...
3,57674,NaN,Bulk Item Disposal,"Learn how to get rid of bulk items. Schedule, ..."
4,57213,NaN,Heat or Hot Water Complaint in a Residential B...,Report a problem with heat or hot water in a r...
5,55351,NaN,Noise from Neighbor,Report a noisy neighbor.
6,55032,NaN,Blocked Driveway,Report a vehicle that is blocking your driveway.
7,54232,NaN,Parking Ticket or Camera Violation Status,Look up information about your parking or came...
8,52887,NaN,Service Request Status,Check the status of a Service Request filed wi...
9,52218,NaN,Apartment Maintenance Complaint,Report a maintenance problem in a privately-ow...


In [4]:
len(call_comps)

6596

In [5]:
len(call_comps[call_comps['cat'].isna()])

2107

In [10]:
call_comps['combined_cats'] = call_comps['cat'].fillna('') \
                      .str.cat(call_comps['subcat'].fillna(''), sep=' ') \
                      .str.cat(call_comps['subsubcat'].fillna(''), sep=' ')

call_comps.head()

,count,cat,subcat,subsubcat,combined_cats
0,84545,Department of Finance,Parking Ticket Lookup - Ticket or Plate Number...,Status of a parking ticket. Must know ticket o...,Department of Finance Parking Ticket Lookup - ...
1,76516,3-1-1 Call Center,Service Request Status - General,Status of most Service Requests filed with 311.,3-1-1 Call Center Service Request Status - Gen...
2,60523,Department of Finance,Property Tax Account Assistance and Bill Infor...,Help with questions about the property tax bil...,Department of Finance Property Tax Account Ass...
3,57674,NaN,Bulk Item Disposal,"Learn how to get rid of bulk items. Schedule, ...",Bulk Item Disposal Learn how to get rid of bu...
4,57213,NaN,Heat or Hot Water Complaint in a Residential B...,Report a problem with heat or hot water in a r...,Heat or Hot Water Complaint in a Residential ...


In [11]:
#clean combined_cats column to prepare for NLTK processing
#lowercase
call_comps['combined_cats_lower'] = call_comps['combined_cats'].str.lower()
#make list
txt_ls = call_comps['combined_cats_lower'].tolist()

In [25]:
import re, string

tmp_list_unique = []

for item in txt_ls:
    
    item = item.replace('sanitation', 'sanitation waste')
    item = item.replace('parks', 'parks recreation')
    item = item.replace('recreation', 'parks recreation')
    item = item.replace('watr', 'water')
    item = item.replace('wats', 'water')
    item = item.replace('wrs', 'waste')
    
    item = item.replace('park maintenance', 'recreation and parks maintenance')
    item = item.replace('park rules', 'recreation and parks rules')
    item = item.replace('park equipment', 'recreation and parks equipment')
    item = item.replace('park lawn', 'recreation and parks lawn')
    item = item.replace('park programs', 'recreation and parks program')
    item = item.replace('park light', 'recreation and parks light')
    item = item.replace('park rangers', 'recreation and parks rangers')
    item = item.replace('park surface', 'recreation and parks surface')
    item = item.replace('rec and park', 'recreation and parks')
    
    item = item.replace('wildlife', 'wildlife animal')
    item = item.replace('pigeon', 'pigeon animal')
    item = item.replace('bats', 'bats animal')
    item = item.replace('dog', 'dog animal')
    item = item.replace('stray', 'stray animal')
    item = item.replace('coyote', 'coyote animal')
    item = item.replace('pot hole', 'pothole street')
    item = item.replace('sinkhole', 'sinkhole pothole street')
    
    item = item.replace('parking', 'parking vehicles')
    item = item.replace('bugs', 'public health bugs pest control')
    itme = item.replace('mosquitoes', 'public health mosquito pest control')
    itme = item.replace('mosquito', 'public health mosquito pest control')
    itme = item.replace('roaches', 'public health roaches pest control')
    item = item.replace('mice', 'public health rats rodent pest control')
    item = item.replace('rodent', 'public health rodent rats pest control')
    item = item.replace('dumping', 'dumping waste')
    item = item.replace('illdump', 'illegal dumping waste')
    item = item.replace('plmb', 'plumbming')
    item = item.replace('haz', 'hazard')
    item = item.replace('cons', 'construction')
    item = item.replace('prmt', 'permit')
    item = item.replace('cnflct', 'conflict')
    item = item.replace('insp', 'inspection')
    item = item.replace('litter', 'litter waste')
    item = item.replace('isd', '')
    item = item.replace('traum counseling', 'trauma counseling')
    item = item.replace('dirty', 'dirty unsanitary')
    item = item.replace('bldgmaint', 'building maintenance')
    item = item.replace('derelict', 'abandonded')
    item = item.replace('blighted', 'abandonded')
    item = item.replace('vacant', 'abandonded')
    item = item.replace('building code', 'inspection code enforcement violation')
    item = item.replace('inspection', 'inspection code enforcement violation')
    item = item.replace('code enforcement', 'inspection code enforcement violation')
    item = item.replace('code violations', 'inspection code enforcement violation')
    item = item.replace('housing violations', 'inspection code enforcement violation')
    item = item.replace('property violations', 'inspection code enforcement violation')
    item = item.replace('codes compliance', 'inspection code enforcement violation')
    item = item.replace('transit', 'transit transportation')
    item = item.replace('transportation', 'transit transportation')
    item = item.replace('tenant', 'tenant housing rental')
    item = item.replace('rental issues', 'tenant housing rental')
    item = item.replace('brush', 'brush debris bulky')
    item = item.replace('loose leaf', 'brush debris bulky')
    item = item.replace('debris', 'brush debris bulky')
    item = item.replace('stlight(s)', 'street light')
    item = item.replace('collectn', 'bulky waste collection')
    item = item.replace('household appliances', 'bulky waste collection')
    item = item.replace('special collections', 'bulky waste collection')
    item = item.replace('property maintenance', 'property maintenance building residential')
    item = item.replace('building residential', 'property maintenance building residential')
    item = item.replace('forestry', 'tree')
    item = item.replace('national grid', 'electricity gas')
    item = item.replace('bicycle', 'bike')
    item = item.replace('botanical', 'tree')
    item = item.replace('property', 'property building')
    item = item.replace('pay station', 'pay station parking meter')
    item = item.replace('general inquiry', 'general information')
    item = item.replace('question', 'general information')
    item = item.replace('general questions', 'general information')
    item = item.replace('general request', 'general information')
    item = item.replace('literature request', 'general information')
    item = item.replace('program information', 'general information')
    item = item.replace('information request', 'general information')
    item = item.replace('records request', 'general information')
    
    item = item.replace('manhole', 'manhole drain')
    item = item.replace('missed collection', 'missed collection waste')
    item = item.replace('roadway', 'roads')
    
    item = item.replace('tax - ', 'tax taxpayer treasury')
    item = item.replace('compliment', 'commendation')
    
    item = item.replace('constructiontruction', 'construction')
    item = item.replace('contructionumber', 'construction')
    
    for number in ['1','2','3','4','5','6','7','8','9','0']:    
        item = item.replace(number,'')
    
    #remove symbols exc NEW CODE INSERTED BY LIZ JOHNSON 11/15/22
    symbols = "!\"\'#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in symbols:
        item = np.char.replace(item, i, '')
    
    tmp_list_unique.append(str(item).strip())

In [26]:
print(tmp_list_unique[0:10])

['department of finance parking vehicles ticket lookup  ticket or plate number known status of a parking vehicles ticket must know ticket or license plate number', 'call center service request status  general status of most service requests filed with', 'department of finance property building tax account assistance and bill information help with general informations about the property building tax bill, also known as the statement of account', 'bulk item disposal learn how to get rid of bulk items schedule, reschedule, cancel, or check the status of a bulk item pickup appointment', 'heat or hot water complaint in a residential building report a problem with heat or hot water in a residential building', 'noise from neighbor report a noisy neighbor', 'blocked driveway report a vehicle that is blocking your driveway', 'parking vehicles ticket or camera violation status look up information about your parking vehicles or camera violations and get payment, dispute, and appeal status', 'serv

In [27]:
stopwords = nltk.corpus.stopwords.words('english')
#remove stopwords
tmp_list_unique = [[word for word in sentence.split() if word.lower() not in stopwords] for sentence in tmp_list_unique]
#join split sentences back
tmp_list_unique = [" ".join(split_sentence) for split_sentence in tmp_list_unique]

In [28]:
print(tmp_list_unique[0:10])

['department finance parking vehicles ticket lookup ticket plate number known status parking vehicles ticket must know ticket license plate number', 'call center service request status general status service requests filed', 'department finance property building tax account assistance bill information help general informations property building tax bill, also known statement account', 'bulk item disposal learn get rid bulk items schedule, reschedule, cancel, check status bulk item pickup appointment', 'heat hot water complaint residential building report problem heat hot water residential building', 'noise neighbor report noisy neighbor', 'blocked driveway report vehicle blocking driveway', 'parking vehicles ticket camera violation status look information parking vehicles camera violations get payment, dispute, appeal status', 'service request status check status service request filed nyc', 'apartment maintenance complaint report maintenance problem privatelyowned residential building'

In [30]:
call_comps['processed_comps'] = tmp_list_unique

In [31]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to calculate sentiment scores for each row in the dataframe
def get_sentiment_scores(row):
    # Use the VADER sentiment analyzer to get a sentiment score for the combined categories text
    sentiment_scores = analyzer.polarity_scores(row['processed_comps'])
    # Return the sentiment score dictionary
    return sentiment_scores

# Apply the function to each row in the dataframe and store the results in a new column
call_comps['sentiment_scores'] = call_comps.apply(get_sentiment_scores, axis=1)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [32]:
call_comps.head(10)

,count,cat,subcat,subsubcat,combined_cats,combined_cats_lower,processed_comps,sentiment_scores
0,84545,Department of Finance,Parking Ticket Lookup - Ticket or Plate Number...,Status of a parking ticket. Must know ticket o...,Department of Finance Parking Ticket Lookup - ...,department of finance parking ticket lookup - ...,department finance parking vehicles ticket loo...,"{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'comp..."
1,76516,3-1-1 Call Center,Service Request Status - General,Status of most Service Requests filed with 311.,3-1-1 Call Center Service Request Status - Gen...,3-1-1 call center service request status - gen...,call center service request status general sta...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,60523,Department of Finance,Property Tax Account Assistance and Bill Infor...,Help with questions about the property tax bil...,Department of Finance Property Tax Account Ass...,department of finance property tax account ass...,department finance property building tax accou...,"{'neg': 0.0, 'neu': 0.876, 'pos': 0.124, 'comp..."
3,57674,NaN,Bulk Item Disposal,"Learn how to get rid of bulk items. Schedule, ...",Bulk Item Disposal Learn how to get rid of bu...,bulk item disposal learn how to get rid of bu...,bulk item disposal learn get rid bulk items sc...,"{'neg': 0.111, 'neu': 0.889, 'pos': 0.0, 'comp..."
4,57213,NaN,Heat or Hot Water Complaint in a Residential B...,Report a problem with heat or hot water in a r...,Heat or Hot Water Complaint in a Residential ...,heat or hot water complaint in a residential ...,heat hot water complaint residential building ...,"{'neg': 0.308, 'neu': 0.692, 'pos': 0.0, 'comp..."
5,55351,NaN,Noise from Neighbor,Report a noisy neighbor.,Noise from Neighbor Report a noisy neighbor.,noise from neighbor report a noisy neighbor.,noise neighbor report noisy neighbor,"{'neg': 0.298, 'neu': 0.702, 'pos': 0.0, 'comp..."
6,55032,NaN,Blocked Driveway,Report a vehicle that is blocking your driveway.,Blocked Driveway Report a vehicle that is blo...,blocked driveway report a vehicle that is blo...,blocked driveway report vehicle blocking driveway,"{'neg': 0.54, 'neu': 0.46, 'pos': 0.0, 'compou..."
7,54232,NaN,Parking Ticket or Camera Violation Status,Look up information about your parking or came...,Parking Ticket or Camera Violation Status Loo...,parking ticket or camera violation status loo...,parking vehicles ticket camera violation statu...,"{'neg': 0.399, 'neu': 0.601, 'pos': 0.0, 'comp..."
8,52887,NaN,Service Request Status,Check the status of a Service Request filed wi...,Service Request Status Check the status of a ...,service request status check the status of a ...,service request status check status service re...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
9,52218,NaN,Apartment Maintenance Complaint,Report a maintenance problem in a privately-ow...,Apartment Maintenance Complaint Report a main...,apartment maintenance complaint report a main...,apartment maintenance complaint report mainten...,"{'neg': 0.412, 'neu': 0.588, 'pos': 0.0, 'comp..."


In [35]:
# Calculate the mean compound sentiment score for each category
mean_sentiment_scores = call_comps.groupby('combined_cats')['sentiment_scores'].apply(lambda x: x.apply(lambda y: y['compound']).mean())